# Modelos de Regresion Espacial SAR

## Autocorrelación Endógena con la variable dependiente

In [12]:
import geopandas as gpd
import pandas as pd
import numpy as np
import libpysal
from libpysal.weights import Queen
from sklearn.preprocessing import StandardScaler
import spreg

In [13]:
gdf = gpd.read_file("https://github.com/algarciach/AnalisisGeoespacial/raw/main/Covid19_model/Data/covid19_municipios_antioquia.gpkg")
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   codigo_municipio     125 non-null    object  
 1   nombre_municipio     125 non-null    object  
 2   codigo_subregion     125 non-null    object  
 3   nombre_subregion     125 non-null    object  
 4   area_municipio       125 non-null    float64 
 5   altitud              125 non-null    float64 
 6   temperatura          125 non-null    float64 
 7   humedad_relativa     125 non-null    float64 
 8   poblacion            125 non-null    int64   
 9   urbanizacion         125 non-null    float64 
 10  densidad             125 non-null    float64 
 11  muertes_covid19      125 non-null    int64   
 12  recuperados_covid19  125 non-null    int64   
 13  cfr                  125 non-null    float64 
 14  geometry             125 non-null    geometry
dtypes: float64(7), 

In [14]:
gdf['density_cfr'] = (gdf['cfr'] / gdf['area_municipio'])
gdf['y'] = np.log(gdf['density_cfr'] + 1)

y = gdf['y'].values.reshape(-1, 1)
independent_vars = ["altitud", "humedad_relativa", "urbanizacion", "densidad"]
X = gdf[independent_vars].values

st = StandardScaler()
X_scaled = st.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=independent_vars)
w = Queen.from_dataframe(gdf)

wx_dict = {}
for var in independent_vars:
    wx_dict[f'w_{var}'] = libpysal.weights.spatial_lag.lag_spatial(w, X_scaled_df[var])

wx_df = pd.DataFrame(wx_dict)
slx_exog = pd.concat([X_scaled_df, wx_df], axis=1)

ols_model = spreg.OLS(gdf['y'].values.reshape((-1, 1)), 
                      slx_exog.values, 
                      name_y='log_density_cfr', 
                      name_x=slx_exog.columns.tolist(),
                      name_w='queen_contiguity')

print(ols_model.summary)

/var/folders/nq/kj34bm5140bgwbktdx1z94cm0000gn/T/ipykernel_1010/3943410554.py:11: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf)


REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :log_density_cfr                Number of Observations:         125
Mean dependent var  :      0.0146                Number of Variables   :           9
S.D. dependent var  :      0.0161                Degrees of Freedom    :         116
R-squared           :      0.2402
Adjusted R-squared  :      0.1878
Sum squared residual:   0.0245561                F-statistic           :      4.5831
Sigma-square        :       0.000                Prob(F-statistic)     :   7.092e-05
S.E. of regression  :       0.015                Log likelihood        :     356.077
Sigma-square ML     :       0.000                Akaike info criterion :    -694.154
S.E of regression ML:      0.0140                Schwarz criterion     :    -668.699

---------------------------------------------------------

In [15]:
spatial_error_model = spreg.GM_Error_Het(y, X_scaled, w=w,
                                         name_y='log_density_cfr_rec',
                                         name_x=independent_vars,
                                         name_w='queen_contiguity')
print(spatial_error_model.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: GM SPATIALLY WEIGHTED LEAST SQUARES (HET)
------------------------------------------------------------
Data set            :     unknown
Weights matrix      :queen_contiguity
Dependent Variable  :log_density_cfr_rec                Number of Observations:         125
Mean dependent var  :      0.0146                Number of Variables   :           5
S.D. dependent var  :      0.0161                Degrees of Freedom    :         120
Pseudo R-squared    :      0.2302
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         0.01565         0.00177         8.82471         0.00000
             altitud         0.00258         

In [22]:
sem = spreg.ML_Error(y, X, w=w, name_y='log_density_cfr_rec', name_x=independent_vars, name_w='queen_contiguity')
print(sem.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ML SPATIAL ERROR (METHOD = full)
---------------------------------------------------
Data set            :     unknown
Weights matrix      :queen_contiguity
Dependent Variable  :log_density_cfr_rec                Number of Observations:         125
Mean dependent var  :      0.0146                Number of Variables   :           5
S.D. dependent var  :      0.0161                Degrees of Freedom    :         120
Pseudo R-squared    :      0.1538
Log likelihood      :    327.1198
Sigma-square ML     :      0.0002                Akaike info criterion :    -644.240
S.E of regression   :      0.0146                Schwarz criterion     :    -630.098

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         

/usr/local/Caskroom/miniforge/base/envs/geo/lib/python3.11/site-packages/spreg/ml_error.py:184: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  res = minimize_scalar(
/usr/local/Caskroom/miniforge/base/envs/geo/lib/python3.11/site-packages/spreg/ml_error.py:563: RuntimeWarning: invalid value encountered in log
  jacob = np.log(np.linalg.det(a))


In [23]:
spatial_lag_model = spreg.GM_Lag(y, X_scaled, w=w,
                                 name_y='log_density_cfr_rec',
                                 name_x=independent_vars,
                                 name_w='queen_contiguity')
print(spatial_lag_model.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :queen_contiguity
Dependent Variable  :log_density_cfr_rec                Number of Observations:         125
Mean dependent var  :      0.0146                Number of Variables   :           6
S.D. dependent var  :      0.0161                Degrees of Freedom    :         119
Pseudo R-squared    :      0.2027
Spatial Pseudo R-squared:  0.1834

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         0.01141         0.00421         2.71028         0.00672
             altitud         0.00161         0.00187         0.86164         0.38889
    humedad_relativa        -0.